In [313]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from boardgamegeek import BGGClient
import re 
import time

bgg = BGGClient()

In [314]:
game_ext = []


## Scraping Function for BGG

In [315]:
def listing(val):
    url = 'https://boardgamegeek.com/browse/boardgame/page/' + str(val)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html5lib")

    table = soup.find(class_ = "table-responsive")

    rows = table.find_all('tr')

    for i in rows:
        test = i.find_all('a', href=True)
        if test[0]['href'] != "/browse/boardgame?sort=rank&sortdir=desc":
            game_ext.append(test[0]['href'])


## Scraping top 5000 Games for game id

In [319]:
for i in range(50):
    listing(i+1)

In [320]:
game_id = []
for i in game_ext:
    x = re.search("boardgame\/(.*?)\/",i)
    game_id.append(x.group(1))

## API Crawl for the 5000 game ids scraped above

In [333]:
cols= ['Name', 'Published', 'Owned', 'Mechanics', 'Categories', 'Min Age', 'Min Players', 'Max Players', 'Min Play Time', 'Max Playtime', 'Complexity [0-5]', 'User Rating']
df2 = pd.DataFrame(columns = cols)

In [334]:
def api_crawl(val):

    g = bgg.game(game_id = int(game_id[val]), marketplace = True, versions = True)
    dat = [g.name,g.year, g.users_owned, ", ".join(g.mechanics), ", ".join(g.categories), g.min_age, g.min_players, g.max_players, g.min_playing_time, g.max_playing_time, g.rating_average_weight, g.rating_average]
    a_series = pd.Series(dat, index = df2.columns)
    return a_series

    

In [337]:
for i in range(4973, len(game_id)):
    df2 = df2.append(api_crawl(i), ignore_index=True)
    ## Sleeping due to rate limits
    time.sleep(2)

df2.to_csv('export.csv')   

In [338]:
df2

,Name,Published,Owned,Mechanics,Categories,Min Age,Min Players,Max Players,Min Play Time,Max Playtime,Complexity [0-5],User Rating
0,Gloomhaven,2017,70842,"Action Queue, Action Retrieval, Campaign / Bat...","Adventure, Exploration, Fantasy, Fighting, Min...",14,1,4,60,120,3.8631,8.78255
1,Pandemic Legacy: Season 1,2015,66534,"Action Points, Cooperative Game, Hand Manageme...","Environmental, Medical",13,2,4,60,60,2.8395,8.61045
2,Brass: Birmingham,2018,31401,"Hand Management, Income, Loans, Market, Networ...","Economic, Industry / Manufacturing, Trains, Tr...",14,2,4,60,120,3.9144,8.67221
3,Terraforming Mars,2016,90525,"Drafting, End Game Bonuses, Hand Management, H...","Economic, Environmental, Industry / Manufactur...",12,1,5,120,120,3.2397,8.43097
4,Twilight Imperium: Fourth Edition,2017,17662,"Action Drafting, Area Majority / Influence, Ar...","Civilization, Economic, Exploration, Negotiati...",14,3,6,240,480,4.2295,8.69360
...,...,...,...,...,...,...,...,...,...,...,...,...
4996,[redacted],2014,1812,"Area Movement, Hand Management, Hidden Roles, ...","Bluffing, Spies/Secret Agents",12,2,6,45,45,2.9250,6.19167
4997,Chase,1985,532,Grid Movement,"Abstract Strategy, Dice",10,2,2,45,45,2.6579,6.81171
4998,1824: Austria-Hungary,2005,205,"Auction/Bidding, Network and Route Building, S...","Economic, Trains",14,3,6,300,300,3.7647,7.79486
4999,Bang! The Dice Game: The Walking Dead,2015,786,"Dice Rolling, Player Elimination, Variable Pla...","Bluffing, Comic Book / Strip, Deduction, Dice,...",8,3,8,15,30,1.3333,6.66621
